## Lab 3. Peak Load Manager

### Introduction

In this notebook we show you how to create another agent on Amazon Bedrock.

This agent will be a peak load manager agent, identifying non-essential processes that can be shifted to off-peak hours.

The following represents the piece of architecture that will be built on this module.

![Architecture](../img/lab3-architecture.png)

### Setup

Firstly, you are going to install boto3 dependencies from pip. Make sure to have version superior of **1.35.45**

In [ ]:
!pip uninstall boto3 botocore awscli --yes

In [ ]:
# Install Dependencies from local package
!pip install ../boto3/botocore-1.35.55-py3-none-any.whl \
    ../boto3/boto3-1.35.55-py3-none-any.whl \
    ../boto3/awscli-1.35.21-py3-none-any.whl --force-reinstall --no-cache

Restart kernel for packages to take effect

In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

Check if your boto3 version is superior than **1.35.45**

In [ ]:
!pip freeze | grep boto3

### Creating Agent

On this section we're going to declare global variables that will be act as helpers during entire notebook and you will start to create your agent.

In [ ]:
import boto3
import os
import sys
import json

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

agent_foundation_model = ['anthropic.claude-3-5-sonnet-20240620-v1:0']
#agent_foundation_model = ['anthropic.claude-3-5-haiku-20241022-v1:0']

In [ ]:
peak_agent_name = f"peak-agent-{account_id}"

peak_lambda_name = f"fn-peak-agent-{account_id}"

peak_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{peak_agent_name}'

dynamodb_table = f"{peak_agent_name}-table"
dynamodb_pk = "customer_id"
dynamodb_sk = "item_id"

dynamoDB_args = [dynamodb_table, dynamodb_pk, dynamodb_sk]


On following section, we're adding `agents.py` on Python path, so the file can be recognized and invoked.

In [ ]:
# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

parent_dir = os.path.dirname(current_dir)
#print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

Now, you're going to import from helper file `agents.py` AgentsForAmazonBedrock helper class.

This class is a helper totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by this class.

Following are methods that you're going to invoke on this lab:

- create_agent: Create a new agent and respective IAM roles
- add_action_group_with_lambda: Create a lambda function and add it as an action group for a previous created agent
- create_agent_alias: Create an alias for this agent
- invoke: Execute agent

In [ ]:
from agent import AgentsForAmazonBedrock

agents = AgentsForAmazonBedrock()

Create the peak load manager agent

In [ ]:
peak_agent_id = agents.create_agent(
    peak_agent_name, 
    """
        You are a peak load manager bot. 
        You can retrieve information from IoT devices, identify process and their peak energy consumption and suggest shifts to off-peak hours.
    """, 
    """
        You are a peak load manager bot. 
        You can retrieve information from IoT devices, identify process and their peak energy consumption and suggest shifts to off-peak hours.
        Resist the temptation to ask the user for input. Only do so after you have exhausted available actions. 
        Never ask the user for information that you already can retrieve yourself through available actions. 

    """,
    agent_foundation_model
)

peak_agent_id

### Creating Lambda

On this block, we're going to generate Lambda function Code and associate it with created agent.

In [ ]:
%%writefile peak_load.py
import os
import boto3
import json
import random

from boto3.dynamodb.conditions import Key, Attr

dynamodb_resource = boto3.resource('dynamodb')
dynamodb_table = os.getenv('dynamodb_table')
dynamodb_pk = os.getenv('dynamodb_pk')
dynamodb_sk = os.getenv('dynamodb_sk')

def get_named_parameter(event, name):
    return next(item for item in event['parameters'] if item['name'] == name)['value']
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def put_dynamodb(table_name, item):
    table = dynamodb_resource.Table(table_name)
    
    resp = table.update_item(
        Key={'customer_id': item['customer_id'],
             'item_id': item['item_id']},
        UpdateExpression='SET #attr1 = :val1',
        ExpressionAttributeNames={'#attr1': 'quota'},
        ExpressionAttributeValues={':val1':  item['quota']}
    )
    return resp

def read_dynamodb(table_name: str, 
                   pk_field: str,
                   pk_value: str,
                   sk_field: str=None, 
                   sk_value: str=None,
                   attr_key: str=None,
                   attr_val: str=None,):
    try:

        table = dynamodb_resource.Table(table_name)
        # Create expression
        if sk_field:
            key_expression = Key(pk_field).eq(pk_value) & Key(sk_field).eq(sk_value)
        else:
            key_expression = Key(pk_field).eq(pk_value)

        if attr_key:
            attr_expression = Attr(attr_key).eq(attr_val)
            query_data = table.query(
                KeyConditionExpression=key_expression,
                FilterExpression=attr_expression
            )
        else:
            query_data = table.query(
                KeyConditionExpression=key_expression
            )
        
        return query_data['Items']
    except Exception:
        print(f'Error querying table: {table_name}.')


def detect_peak(customer_id):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk, 
                         customer_id, 
                         attr_key="peak", attr_val="True")

def detect_non_essential_processes(customer_id):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk, 
                         customer_id,
                         attr_key="essential", attr_val="False")

                
def redistribute_allocation(customer_id, item_id, quota):
    item = {
        'customer_id': customer_id,
        'item_id': item_id,
        'quota': quota
    }
    resp = put_dynamodb(dynamodb_table, item)
    return "Item {} has been updated. New quota: {}".format(item_id, quota)


def lambda_handler(event, context):
    print(event)
    
    # name of the function that should be invoked
    function = event.get('function', '')

    # parameters to invoke function with
    parameters = event.get('parameters', [])
    
    customer_id = get_named_parameter(event, "customer_id")

    if function == 'detect_peak':    
        result = detect_peak(customer_id)
    elif function == 'detect_non_essential_processes':    
        result = detect_non_essential_processes(customer_id)
    elif function == 'redistribute_allocation':    
        item_id = get_named_parameter(event, "item_id")
        quota = get_named_parameter(event, "quota")
        result = redistribute_allocation(customer_id, item_id, quota)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

Add lambda as an action group for this agent and prepare it.

In [ ]:
functions_def = [
    {
        "name": "detect_peak",
        "description": """detect consumption peak during current month""",
        "parameters": {
                        "customer_id": {
                            "description": "The ID of the customer",
                            "required": True,
                            "type": "string"
                        }
                    }
    },
    {
        "name": "detect_non_essential_processes",
        "description": """detect items that are causing peaks""",
        "parameters": {
                        "customer_id": {
                            "description": "The ID of the customer",
                            "required": True,
                            "type": "string"
                        }
                    }
    },
    {
        "name": "redistribute_allocation",
        "description": """reduce/increase allocated quota for a specific 
                            item during current month""",
        "parameters": {
                        "customer_id": {
                            "description": "The ID of the customer",
                            "required": True,
                            "type": "string"
                        },
                        "item_id": {
                            "description": "Item that will be updated",
                            "required": True,
                            "type": "string"
                        },
                        "quota": {
                            "description": "new quota",
                            "required": True,
                            "type": "string"
                        }
                    }
    }
]

In [ ]:
resp = agents.add_action_group_with_lambda(
            agent_name=peak_agent_name, 
            lambda_function_name=peak_lambda_name, 
            source_code_file="peak_load.py", 
            agent_functions=functions_def,
            agent_action_group_name="peak_load_actions", 
            agent_action_group_description="Function to get usage, peaks, redistribution for a user",
            dynamo_args=dynamoDB_args
            )

Load data into DynamoDB Table to run some tests

In [ ]:
with open("3_peak_sample_data.json") as f:
    table_items = [json.loads(line) for line in f]

agents.load_dynamodb(dynamodb_table, table_items)

Testing that data was loaded on DynamoDB

In [ ]:
#resp = agents.query_dynamodb(dynamodb_table, dynamodb_pk, '1')
resp = agents.query_dynamodb(dynamodb_table, dynamodb_pk, '1', dynamodb_sk, "1")
resp

### Create alias

Create an alias that will be used further on multi-agent collaborator feature:

In [ ]:
agents_boto3 = boto3.client('bedrock-agent',
                            region_name=region)

peak_agent_alias_resp = agents_boto3.create_agent_alias(agentId=peak_agent_id, 
                                                              agentAliasName='v1')

### Invoking Agent

Now, let's run some tests on agent to make sure it's working:

In [ ]:
%%time
print(agents.invoke("What's causing my peak load? My id is 2", 
                    peak_agent_id)
)

In [ ]:
%%time
print(agents.invoke("Is it possible to optimize my consumption? My id is 1", 
                    peak_agent_id)
)

In [ ]:
%%time
print(agents.invoke("""Is it possible to change quota allocation? My id is 2, 
                        my item is 5 and new quota is 100""", 
                    peak_agent_id)
)

Store environment variables to be used on next notebooks.

In [ ]:
peak_agent_arn = agents.get_agent_arn_by_name(peak_agent_name)
peak_agent_alias_arn = peak_agent_alias_resp['agentAlias']['agentAliasArn']
peak_agent_alias_id = peak_agent_alias_resp['agentAlias']['agentAliasId']
peak_dynamodb = dynamodb_table

%store peak_agent_arn
%store peak_agent_alias_arn
%store peak_agent_alias_id
%store peak_lambda_name
%store peak_agent_name
%store peak_agent_id
%store peak_dynamodb

In [ ]:
peak_agent_arn, peak_agent_alias_arn, peak_agent_alias_id

### Clean Up

In [ ]:
agents.delete_lambda(peak_lambda_name, dynamoDB_table=dynamodb_table)

In [ ]:
agents_boto3.delete_agent_alias(agentAliasId=peak_agent_alias_id, agentId=peak_agent_id)

In [ ]:
agents.delete_agent(peak_agent_name)